In [55]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path) 
    
from warehouse import DataWarehouse


/Users/keith/Desktop/NUS/Y2S2/IS3107/Project/ezproperty/db


In [208]:
db = DataWarehouse()

dataset = db.query('''
    SELECT district, floorRangeStart, floorRangeEnd, area, transactionDate, resale, price FROM main__PropertyTransaction
    LIMIT 5000
''')


# Data pre-processing to convert all to float and standardise magnitude
df = pd.DataFrame(dataset)
for column in df.columns:
    if column == "transactionDate":
        df[column] = pd.to_datetime(df[column])
        df[column] = (df[column].max() - df[column]) / np.timedelta64(1,'Y')
    if column == "price": # price is in millions
        df[column] = df[column].astype(float) / 1e6
    if column == "area": # area is in 100 square feet
        df[column] = df[column].astype(float) / 100
    else:
        df[column] = df[column].astype(float)
print(df.tail())
print(df.dtypes)

Connecting to 'localhost' with user 'root'
Database | Using database is3107g6.
Database | Query executed successfully.
      district  floorRangeStart  floorRangeEnd  area  transactionDate  resale  \
4995       3.0              6.0           10.0  1.06         1.579772     0.0   
4996       3.0             16.0           20.0  0.76         1.579772     0.0   
4997       3.0             21.0           25.0  1.16         1.579772     0.0   
4998       3.0              1.0            5.0  1.16         1.579772     0.0   
4999       3.0             11.0           15.0  1.59         1.579772     0.0   

        price  
4995  2.84946  
4996  2.25270  
4997  3.30000  
4998  3.05700  
4999  4.26430  
district           float64
floorRangeStart    float64
floorRangeEnd      float64
area               float64
transactionDate    float64
resale             float64
price              float64
dtype: object


In [353]:
train = df.sample(frac=0.9,random_state=200)
test = df.drop(train.index)

X_train, Y_train = train[[column for column in df.columns if column != 'price']], train['price']
X_test, Y_test = test[[column for column in df.columns if column != 'price']], test['price']

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(4500, 6) (500, 6) (4500,) (500,)


In [384]:
# Build model
def build_mlp_model():
  model = keras.Sequential()
  model.add(Dense(6, activation='relu', input_shape=(6,)))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='linear'))
  return model

mlp_model = build_mlp_model()
mlp_model.summary()

# Build model
# def build_mlp_model():
#   model = keras.Sequential()
#   model.add(Dense(6, activation='relu', input_shape=(6,)))
#   # model.add(Dropout(0.1))
#   model.add(Dense(4, activation='relu'))
#   # model.add(Dropout(0.1))
#   model.add(Dense(4, activation='relu'))
#   # model.add(Dropout(0.1))
#   model.add(Dense(1, activation='linear'))
#   return model

# mlp_model = build_mlp_model()
# mlp_model.summary()

Model: "sequential_74"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_331 (Dense)           (None, 6)                 42        
                                                                 
 dense_332 (Dense)           (None, 64)                448       
                                                                 
 dense_333 (Dense)           (None, 64)                4160      
                                                                 
 dense_334 (Dense)           (None, 64)                4160      
                                                                 
 dense_335 (Dense)           (None, 1)                 65        
                                                                 
Total params: 8,875
Trainable params: 8,875
Non-trainable params: 0
_________________________________________________________________


In [385]:
# Compile the model
mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='mae', metrics=['mae'])
# mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])
# mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [386]:
mlp_history = mlp_model.fit(X_train, Y_train, epochs=10, batch_size=16)

Epoch 1/10
282/282 [==============================] - 2s 6ms/step - loss: 1.0291 - mae: 1.0291
Epoch 2/10
282/282 [==============================] - 2s 6ms/step - loss: 0.7092 - mae: 0.7092
Epoch 3/10
282/282 [==============================] - 2s 6ms/step - loss: 0.6313 - mae: 0.6313
Epoch 4/10
282/282 [==============================] - 2s 6ms/step - loss: 0.5820 - mae: 0.5820
Epoch 5/10
282/282 [==============================] - 2s 7ms/step - loss: 0.5414 - mae: 0.5414
Epoch 6/10
282/282 [==============================] - 2s 7ms/step - loss: 0.5154 - mae: 0.5154
Epoch 7/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4858 - mae: 0.4858
Epoch 8/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4457 - mae: 0.4457
Epoch 9/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4522 - mae: 0.4522
Epoch 10/10
282/282 [==============================] - 2s 6ms/step - loss: 0.4396 - mae: 0.4396


In [387]:
loss, acc = mlp_model.evaluate(X_test, Y_test)
print(X_test.shape)
print(f'test loss is {loss}')
print(f'test accuracy is {acc}')

16/16 [==============================] - 0s 8ms/step - loss: 0.4118 - mae: 0.4118
(500, 6)
test loss is 0.41177693009376526
test accuracy is 0.41177693009376526
